In [1]:
import pandas as pd
import datetime as dt
from datetime import date, datetime
import re
import os
import glob

import matplotlib.pyplot as plt
import seaborn as sns

from config import proj_sheet, datapath, credential_path, drivesheet_url
import gspread
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file(credential_path, scopes=scopes)
gc = gspread.authorize(credentials)

today = date.today() 
week_ago = today - dt.timedelta(days=7)
today = today.strftime("%d%m%Y")
week_ago = week_ago.strftime("%Y-%m-%d")

# passive + ema_data
filepath = datapath + f"export_{today}.csv"
datapath1 = datapath + f"export_tiki_{today}/"


## 1. Import data

### 1.1 Import epoch level passive + GPS data

In [2]:
file_pattern = os.path.join(datapath1, "epoch_part*.csv")
file_list = glob.glob(file_pattern)
file_list.sort()

In [3]:
# Sort the file list to ensure it's in the correct order, if necessary
df_complete = pd.concat((pd.read_csv(f, encoding="latin-1", low_memory=False) for f in file_list), ignore_index=True)
df_complete["customer"] = df_complete.customer.str.split("@").str.get(0)
df_complete["customer"] = df_complete["customer"].str[:4]

In [4]:
df_complete["startTimestamp"] = pd.to_datetime(df_complete["startTimestamp"],unit='ms')
df_complete["createdAt"] = pd.to_datetime(df_complete["createdAt"],unit='ms')

df_complete["startTimestamp_day"] = df_complete.startTimestamp.dt.strftime('%Y/%m/%d')
df_complete["createdAt_day"] = df_complete.startTimestamp.dt.strftime('%Y/%m/%d')

df_complete["startTimestamp_hour"] = df_complete.startTimestamp.dt.hour
df_complete["createdAt_hour"] = df_complete.startTimestamp.dt.hour

### 1.2 Import passive data

### Check location data

In [5]:
# Location data
df_loc_complete = df_complete[df_complete.type.isin(["Latitude", "Longitude"])]
df_loc_complete = df_loc_complete[["customer", "startTimestamp", "type", "doubleValue", 
                           "timezoneOffset"]]
df_loc_complete["startTimestamp"] = (pd.to_datetime(df_loc_complete["startTimestamp"],unit='ms'))

df_loc_complete = df_loc_complete.groupby("customer")[["startTimestamp"]].max().rename_axis(None, axis=1).reset_index()
df_loc_complete.rename(columns={"startTimestamp":"last_day_GPS"}, inplace=True)


In [6]:
# Passive data
df_pd_complete = df_complete[df_complete.type.isin(["HeartRate", "AtrialFibrillationDetection", "RawECGVoltage", 
                                                   "ActiveBurnedCalories", 'SleepAwakeBinary','SleepBinary'])]
                                                    
df_pd_complete = df_pd_complete[["customer", "startTimestamp", "type", "doubleValue", "timezoneOffset"]]
df_pd_complete["startTimestamp"] = (pd.to_datetime(df_pd_complete["startTimestamp"],unit='ms'))

df_pd_complete = df_pd_complete.groupby("customer")[["startTimestamp"]].max().rename_axis(None, axis=1).reset_index()
df_pd_complete.rename(columns={"startTimestamp":"last_day_passive"}, inplace=True)

### 1.3 Import montoring data

In [7]:
# project management data
df_sheet = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{proj_sheet}/export?format=csv")


In [8]:
df_monitoring = df_sheet.copy()

In [9]:
df_monitoring = df_monitoring[['FOR_ID', 'EMA_ID', 'Pseudonym', 'Studienversion', 'Status',
       'Besonderes', 'Start EMA Baseline', 'Ende EMA Baseline',
       'Terminpräferenz', 'Termin 1. Gespräch',  'Telefonat stattgefunden?',
       'Reminder-Mail Wechsel verschickt?', 'Baseline T20 Update verschickt?', 'Freischaltung/ Start EMA T20',
       'Ende EMA T20', 'Post T20 Update verschickt?',
       'Studienende/ Dropout Mail verschickt?']]

In [10]:
df_monitoring.rename(columns = {"Pseudonym": "customer", "EMA_ID": "ema_id", "Status": "status",
                                "Studienversion":"study_version", "FOR_ID":"for_id", 
                           "Start EMA Baseline": "ema_base_start", "Ende EMA Baseline": "ema_base_end", 
                           "Freischaltung/ Start EMA T20": "ema_t20_start","Ende EMA T20":"ema_t20_end", 
                               "Termin 1. Gespräch": "first_call_date"}, inplace=True)

In [11]:
df_monitoring["customer"] = df_monitoring["customer"].str[:4]
df_active = df_monitoring.copy()
df_active = df_active[["customer", "ema_id", "ema_base_end", "ema_base_start", "study_version", "for_id", "status"]]
df_active["for_id"] = df_active.for_id.str.strip()

### 1.5 Import EMA data

In [12]:
# load data
session = pd.read_csv(datapath1 + "questionnaireSession.csv")

In [13]:
# session data
session["user"] = session["user"].str[:4]
session.rename(columns = {"user":"customer","completedAt": "quest_complete", "createdAt": "quest_create", "expirationTimestamp": "quest_expir"}, inplace=True)
session["quest_create"] = (pd.to_datetime(session["quest_create"],unit='ms'))
session["quest_complete"] = (pd.to_datetime(session["quest_complete"],unit='ms'))
df_sess = session[["customer", "sessionRun", "quest_create", "quest_complete", "study"]]

In [14]:
# count number of completed EMA beeps in first phase
df_sess1 = df_sess.loc[df_sess.study.isin([24,25])]
sess_count1 = df_sess1.dropna(subset=["quest_complete"]).groupby("customer")["quest_complete"].size()\
.reset_index()
sess_count1 = sess_count1.rename(columns = {"quest_complete":"nquest_EMA1"})

In [15]:
# count number of completed EMA beeps in second phase
df_sess2 = df_sess.loc[df_sess.study.isin([33,34])]
sess_count2 = df_sess2.dropna(subset=["quest_complete"]).groupby("customer")["quest_complete"].size()\
.reset_index()
sess_count2 = sess_count2.rename(columns = {"quest_complete":"nquest_EMA2"})

## 2. Merge dataframes

In [16]:
# Merge monitoring and passive
df_merged = pd.merge(df_pd_complete, df_active, on="customer", how="outer")

In [17]:
# Merge monitoring and EMA
df_merged = pd.merge(df_merged, sess_count1, on="customer", how="outer")
df_merged = pd.merge(df_merged, sess_count2, on="customer", how="outer")

In [18]:
df_merged = pd.merge(df_merged, df_loc_complete, on="customer", how="outer")
df_merged.to_csv(f"data_compliance_{today}.csv")

In [19]:
df_merged = df_merged[df_merged['status'].isin(['Erhebung_1_aktiv', 'Post_Erhebung_1', 'Post_Erhebung_2','Erhebung_2_aktiv'])]


## 3. Check missing data

In [20]:
# get structure of the google spreadsheet
df = pd.read_csv("https://docs.google.com/spreadsheets/d/1z8LZJBBMzzAmiXIS47X8SLk-zSMwDIXSKPit4IlmfuE/export?format=csv&gid=1512138040")
df = df.head(0)

In [21]:
users_to_remove = ["FOR11001", "FOR11034", "FOR13023", "FOR14013", "FOR14064"]
remove_scanwatch = ["FOR14029", "FOR14055"]
remove_gps = ["FOR13013", "FOR14014", "FOR13019"]

In [22]:
df_merged = df_merged[~df_merged['for_id'].isin(users_to_remove)]


In [23]:
# no pd since 7 days 

list_no_pd = df_merged.loc[(df_merged.last_day_GPS > week_ago) & (df_merged.last_day_passive < week_ago)]["for_id"].tolist()
list_no_pd = [string for string in list_no_pd if string not in remove_scanwatch]


In [24]:
# no gps for > 7 days 
list_no_gps = df_merged.loc[(df_merged.last_day_GPS < week_ago)& (df_merged.last_day_passive > week_ago)]["for_id"].tolist()
list_no_gps = [string for string in list_no_gps if string not in remove_gps]


In [25]:
# no gps and no pd for > 7 days

list_no_gpspd = df_merged.loc[(df_merged.last_day_GPS < week_ago) & (df_merged.last_day_passive < week_ago)]["for_id"].tolist()
list_no_gpspd = [string for string in list_no_gpspd if string not in users_to_remove]


In [26]:
# no gps at all 

list_no_gps_at_all = df_merged.loc[df_merged.last_day_GPS.isna()].for_id.tolist()
list_no_gps_at_all = [string for string in list_no_gps_at_all if string not in remove_gps]


In [27]:

# Append each list to the DataFrame
date_today = datetime.today().date()
for column_name, entries in zip(['no_pd', 'no_gps', 'no_gpspd', 'no_gps_at_all'], 
                                [list_no_pd, list_no_gps, list_no_gpspd, list_no_gps_at_all]):
    # Create a new DataFrame for the current list
    temp_df = pd.DataFrame(entries, columns=[column_name])
    
    # Add the "Datum" column with today's date
    temp_df['Datum'] = date_today
    
    # Since we're appending column-wise, we align other columns by setting them to NaN
    # This step ensures the DataFrame has all the necessary columns
    for col in df.columns:
        if col not in temp_df.columns:
            temp_df[col] = ""
    
    # Concatenate the new DataFrame to the original DataFrame
    df = pd.concat([df, temp_df], ignore_index=True)

# Reorder df columns to match the original order, if necessary
df = df[['Datum', 'no_pd', 'no_gps', 'no_gpspd', 'data_deleted', 
         'no_gps_at_all', 'Status', 'Smartphone', 'Grund', 
         'Grund (frei)', 'Unnamed: 10']]

## 4. Create Monitoring Alerts

In [28]:
def convert_date(date_str):
# Remove the day of the week by splitting on the comma and taking the second part
    date_part = date_str.split(", ")[1]
    return date_part

# Apply the conversion function to preprocess the dates
df_monitoring['first_call_date'] = df_monitoring['first_call_date'].apply(convert_date)

# Convert the preprocessed date strings to datetime objects
df_monitoring['first_call_date'] = pd.to_datetime(df_monitoring['first_call_date'], format="%d.%m.%Y")

date_columns = ["ema_base_end", "ema_base_start", "ema_t20_start", "ema_t20_end", "first_call_date"]
df_monitoring = df_monitoring.copy()

# Convert multiple columns to datetime'
for col in date_columns:
    df_monitoring[col] = pd.to_datetime(df_monitoring[col], errors='coerce', dayfirst=True).dt.date

In [29]:
# Create a timestamp for today
today = pd.Timestamp("today")

# Calculate the day of the week for today, where Monday is 0 and Sunday is 6
today_day_of_week = today.dayofweek

In [30]:
# Use today's date, but focus only on the date part
today = pd.Timestamp("today").date()


# Calculate the most recent Monday as the start of the week
week_start_date = today - pd.Timedelta(days=today.weekday())

# Loop through each day of the week from Monday to Friday
for day in range(7):  # Monday to Friday
    target_date = week_start_date + pd.Timedelta(days=day)

    # Format the date to include the weekday name and day.month
    formatted_date = target_date.strftime("%A %d.%m")  # e.g., "Monday 12.02"

    # Initialize message list
    messages = []

    # Assuming the filtering logic is already correctly implemented

    # Debug prints to check if conditions are met (example messages for illustration)
    
    onboarding_ids = df_monitoring[df_monitoring['first_call_date'] == target_date]['for_id'].tolist()      
    
    reminder_ids = df_monitoring[(df_monitoring['ema_base_end'] == (target_date + pd.Timedelta(days=1))) &
                                 (df_monitoring['study_version'].isin(['Kurz', 'Kurz (Wechsel/Abbruch)']))]['for_id'].tolist()

    baseline_ended_ids = df_monitoring[(df_monitoring['ema_base_end'] == target_date - pd.Timedelta(days=1)) &
                                       (df_monitoring['study_version'].isin(['Lang', 'Lang(Wechsel)']))]['for_id'].tolist()
    
    study_finished_ids = df_monitoring[(df_monitoring['ema_base_end'] == target_date) &
                                       (df_monitoring['study_version'].isin(['Kurz', 'Kurz (Wechsel/Abbruch)']))]['for_id'].tolist()

    t20_ended_ids = df_monitoring[df_monitoring['ema_t20_end'] == (target_date - pd.Timedelta(days=1))]['for_id'].tolist()
    
    t20_start_ids = df_monitoring[df_monitoring['ema_t20_start'] == target_date]['for_id'].tolist()


    if onboarding_ids:
        messages.append(f"Onboarding Call: {onboarding_ids}")
    if reminder_ids:
        messages.append(f"Kurzversion Wechsel Reminder: {reminder_ids}")
    if baseline_ended_ids:
        messages.append(f"Baseline Ende Lang (Status ändern): {baseline_ended_ids}")
    if t20_start_ids:
        messages.append(f"T20 Start (Status ändern): {t20_start_ids}")
    if t20_ended_ids:
        messages.append(f"T20 Ende (Status ändern): {t20_ended_ids}")
    if study_finished_ids:
        messages.append(f"Kurzversion Ende: {study_finished_ids}")

    # Print the formatted date along with the actions
    if messages:
        print(f"{formatted_date}: ", "; ".join(messages))
    else:
        print(f"{formatted_date}: No actions needed")



Monday 29.04:  Kurzversion Ende: ['FOR14106', 'FOR14107']
Tuesday 30.04:  Onboarding Call: ['FOR11072', 'FOR13044 ', 'FOR14112']; Baseline Ende Lang (Status ändern): ['FOR11070']
Wednesday 01.05:  Kurzversion Wechsel Reminder: ['FOR11071']
Thursday 02.05:  Onboarding Call: ['FOR11074 ']; Kurzversion Wechsel Reminder: ['FOR14110', 'FOR14109']; Baseline Ende Lang (Status ändern): ['FOR14104', 'FOR14105']; T20 Ende (Status ändern): ['FOR12010']; Kurzversion Ende: ['FOR11071']
Friday 03.05:  Kurzversion Ende: ['FOR14110', 'FOR14109']
Saturday 04.05:  Baseline Ende Lang (Status ändern): ['FOR12043']
Sunday 05.05:  Kurzversion Wechsel Reminder: ['FOR14111 ', 'FOR11072']


## 5. Export missing data to google sheets

In [31]:
missing_data = df.applymap(str).values.tolist()

In [32]:
# open a google sheet
gs = gc.open_by_url("https://docs.google.com/spreadsheets/d/1z8LZJBBMzzAmiXIS47X8SLk-zSMwDIXSKPit4IlmfuE")
# select a work sheet from its name
worksheet1 = gs.worksheet('Datenqualität')

In [33]:

next_row = len(worksheet1.col_values(1)) + 1  # Assuming column A has index 1

# Construct the range string where you want to start appending data
# For example, if starting from column A and the next available row is 10, the range would be 'A10'
start_range = f'A{next_row}'

# Use the update method to append data starting from the specified cell
worksheet1.update(start_range, missing_data)

/var/folders/ys/gx19nmhj6v30dlkkh5nfp5xh0000gn/T/ipykernel_28836/4283573234.py:8: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet1.update(start_range, missing_data)


{'spreadsheetId': '1z8LZJBBMzzAmiXIS47X8SLk-zSMwDIXSKPit4IlmfuE',
 'updatedRange': "'Datenqualität'!A779:K810",
 'updatedRows': 32,
 'updatedColumns': 11,
 'updatedCells': 352}

In [34]:
df_missing = pd.read_csv("https://docs.google.com/spreadsheets/d/1z8LZJBBMzzAmiXIS47X8SLk-zSMwDIXSKPit4IlmfuE/export?format=csv&gid=1512138040")


In [35]:
# Melt the DataFrame
df_long = df_missing.melt(id_vars=["Datum"], value_vars=["no_pd", "no_gps", "no_gpspd"], var_name="condition", value_name="for_id")

# Drop rows where ID is NaN
df_long = df_long.dropna(subset=["for_id"])

# Count occurrences of each ID per condition
df_count = df_long.groupby("for_id")["condition"].value_counts().unstack(fill_value=0)

# Optionally, rename columns back to original condition names for clarity
df_count.columns = ["no_pd_count", "no_gps_count", "no_gpspd_count"]

In [36]:
df_count["missing_sum"] = df_count.no_pd_count + df_count.no_gps_count + df_count.no_gpspd_count

In [37]:
df_count["missing_relative"] = df_count.missing_sum / df_long.Datum.nunique()

In [38]:
df_count = pd.merge(df_active, df_count, on = "for_id", how="outer")